In [1]:
import arcpy
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

strFilterTime = "2022-08-24T00:00:00.000+00:00"

In [2]:
from arcgis import GIS
gis = GIS()
item = gis.content.get("89d4e14f9c24462eaf9d04e12e3276e3")
flayer = item.tables[0]


In [3]:
# create a Spatially Enabled DataFrame object
sdf = pd.DataFrame.spatial.from_layer(flayer)
sdf


,OBJECTID,GeoType,GeoID,CommentText,Email,TimeStampText
0,1,TAZ,,test,williamrh@gmail.com,None
1,2,TAZ,,test2,williamrh@gmail.com,None
2,3,TAZ,None,Test343,williamrh@gmail.com,None
3,4,CITYAREA,None,Test,williamrh@gmail.com,2022-06-16T21:57:20.884Z
4,5,CITYAREA,None,and another,williamrh@gmail.com,2022-06-16T22:01:14.749Z
...,...,...,...,...,...,...
287,309,TAZ,2047,Development agreement includes 211 Single Fami...,bterry@herriman.org,2022-08-24T22:16:06.833Z
288,310,TAZ,2022,Approximately 1700 housing units planned in th...,bterry@herriman.org,2022-08-24T22:29:22.051Z
289,311,TAZ,2034,250 Single Family Units approved in this TAZ,bterry@herriman.org,2022-08-24T22:38:04.360Z
290,312,TAZ,2036,Total of 540 Units planned to be built be 2030...,bterry@herriman.org,2022-08-24T22:42:02.035Z


In [4]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [5]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('streetlight-temp-analysis-1374f860ac96.json', scope)
gc = gspread.authorize(credentials)

In [6]:
spreadsheet_key = '1Sd1L0wTbd079zsMQGN2n0tyyugmsyffM2v97MhzcRUg'
book = gc.open_by_key(spreadsheet_key)
#worksheet = book.worksheet("Sheet1")
#table = worksheet.get_all_values()
#display(table)

In [7]:
##Convert table data into a dataframe
#df = pd.DataFrame(table[1:], columns=table[0])

##Only keep columns we need
#df = df[['Order', 'Candidate', 'Position', 'Start Date', 'End Date', 'Years of Experience', 'Points']]

##Convert number strings to floats and ints
#df = df.apply(pd.to_numeric, errors='ignore')

##Convert date strings to datetime format
#df['End Date'] = pd.to_datetime(df['End Date'],infer_datetime_format=True)
#df['Start Date'] = pd.to_datetime(df['Start Date'],infer_datetime_format=True)

#df

In [8]:
#agoldatatoappend['GeoID'].str.contains("None")

In [9]:
agoldatatoappend = sdf[['GeoType','GeoID','CommentText','Email','TimeStampText']]
agoldatatoappend['TimeStamp'] = pd.to_datetime(agoldatatoappend['TimeStampText'])
#agoldatatoappend = agoldatatoappend[(agoldatatoappend['GeoID'].str.contains("None")) | (agoldatatoappend['GeoID']!="")]

# limit to only after a certain date, in case google sheet was updated
agoldatatoappend = agoldatatoappend[agoldatatoappend['TimeStamp']> strFilterTime]
agoldatatoappend = agoldatatoappend.drop(columns='TimeStamp')
agoldatatoappend['Accessed from AGOL'] = pd.to_datetime("now")
agoldatatoappend

,GeoType,GeoID,CommentText,Email,TimeStampText,Accessed from AGOL
287,TAZ,2047,Development agreement includes 211 Single Fami...,bterry@herriman.org,2022-08-24T22:16:06.833Z,2022-08-30 17:30:30.876048
288,TAZ,2022,Approximately 1700 housing units planned in th...,bterry@herriman.org,2022-08-24T22:29:22.051Z,2022-08-30 17:30:30.876048
289,TAZ,2034,250 Single Family Units approved in this TAZ,bterry@herriman.org,2022-08-24T22:38:04.360Z,2022-08-30 17:30:30.876048
290,TAZ,2036,Total of 540 Units planned to be built be 2030...,bterry@herriman.org,2022-08-24T22:42:02.035Z,2022-08-30 17:30:30.876048
291,TAZ,2033,There should be some jobs here City Hall exist...,bterry@herriman.org,2022-08-24T22:44:22.794Z,2022-08-30 17:30:30.876048


In [10]:
from datetime import datetime

now = pd.Timestamp.now()

wks_name = now.strftime("%Y-%m-%d")
display(wks_name)

'2022-08-30'

In [11]:
from df2gspread import df2gspread as d2g

d2g.upload(agoldatatoappend, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet '2022-08-30' id:43300438>